In [1]:
import pandas as pd
import glob
import numpy as np
import sys
sys.path.append("../scripts")
from badfiles import BadFiles

## Exploracion errores en los archivos
Exploraremos los datos y sus headers en bruto, verificando que:
* las columnas esten bien nombradas y en el orden esperado
* no existan datos vacios entre lineas
* mediciones inconsistentes o fisicamente imposibles como:
    * velocidades de viento, direcciones de viento, radiacion o precipitacion negativas
    * presion atmosferica de 0 o inferior
* el archivo leido no contenga solo 1 fila (es un dato erroneo)
* el archivo leido tenga el formato correcto (que se pueda leer)
* archivos dentro de una estacion/carpeta presentan distintos codigos de estacion

In [2]:
raw_data_path = "../data/PAULA"

bf = BadFiles(raw_data_path)
bf.check_all()

:::::: ARICA Y PARINACOTA ::::::
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/ARICA Y PARINACOTA/COLONIA JFLLUTA BAJO/Coloniafjllutabajo-01-01-2010-30-06-2010.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/ARICA Y PARINACOTA/SOCOROMA/Socoroma 01-01-2010-30-06-2010.xls
ERROR.file.../data/PAULA/ARICA Y PARINACOTA/SOCOROMA/Socoroma. 01-01-09-31-06-2009.xls
ERROR.file.../data/PAULA/ARICA Y PARINACOTA/SOCOROMA/Socoroma. 01-07-09 31-12-09.xls
ERROR.file.../data/PAULA/ARICA Y PARINACOTA/TRUFA AZAPA MEDIO/TrufaAzapaMedio.01-01-2009-30-06-2009.xls
ERROR.file.../data/PAULA/ARICA Y PARINACOTA/TRUFA AZAPA MEDIO/TrufaAzapaMedio.01-07-2009-31-12-2009.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/ARICA Y PARINACOTA/TRUFA AZAPA MEDIO/TrufaAzapaMedio01-01-2010-30-06-2010.xls
:::::: ATACAMA ::::::
ERROR.notUniqueCodeInStation.../data/PAULA/ATACAMA\ATACAMA
ERROR.NoResultFile.../data/PAULA/ATACAMA/BODEGA/Bodega01-01-2009-30-06-2009.xls
ERROR.file.../data/PAULA/ATACAMA/BODEGA/Bodega01-07-2009-31-12-2009.xls
ER

INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/CENKIWI/Cenkiwi-01-01-2010-30-06-2009.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/CENKIWI/Cenkiwi-01-01-2010-30-06-2010.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/CENKIWI/Cenkiwi-01-07-2009-31-12-2009.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/CORDILLERILLA/Cordillerilla-01-01-2009-30-06-2009-.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/CORDILLERILLA/Cordillerilla-01-01-2010-30-06-2010-.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/CORDILLERILLA/Cordillerilla-01-07-2009-31-12-2009-.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/CORONEL DEL MAULE/Coroneldelmaule-01-01-2010-30-06-2010.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/GUAICO TRES/Guaicotres-01-01-2009-30-06-2009.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/GUAICO TRES/Guaicotres-01-01-2010-30-06-2010.xls
INCONCISTENCY.ZeroOrNegPAtm.../data/PAULA/DEL MAULE/GUAICO TRES/Guaicotres-01-07-20

In [3]:
bf.save("../data")
print("total de archivos excel:", bf.counter)
print("archivos excel con problemas:", bf.counter_bads)
print(len(bf.files), len(bf.errors))

total de archivos excel: 497
archivos excel con problemas: 146
149 149


### Posibles correcciones
* Para las tablas con malos headers basta con renombrarlos manualmente
* Para las tablas con NaN, tenemos 2 opciones:
    * OPCION 1: borrar la fila especifica que contiene el dato NaN, esto puede causar series de tiempo irregulares
    * OPCION 2: interpolar el dato perdido (lo mas facil seria una interponacion lineal entre el dato anterior y el siguiente, si es el ultimo borrarlo)
* para los casos en datos malos, manualmente hay que identificar los casos, si son pocos se borran dichos datos asociados, si son muchos, se descarta toda la estacion.
* algunas estacion no piden todas las variables que esperamos. Estas estaciones las descartare directamente

##### por ahora es recomendable evitar estos excels para que realicen sus exploraciones